In [ ]:
!pip install langchain langchain-community langchain-groq -q
!pip install groq PyMuPDF faiss-cpu sentence-transformers -q

import os, fitz, faiss, numpy as np
from kaggle_secrets import UserSecretsClient
from langchain_groq import ChatGroq
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer

os.environ["GROQ_API_KEY"] = UserSecretsClient().get_secret("GROQ_API_KEY")
llm = ChatGroq(model_name="llama-3.3-70b-versatile")

def extract_text(path): return "\n".join(page.get_text() for page in fitz.open(path))
text = extract_text("/kaggle/input/ipc-pdf/IPC_186045.pdf")

splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = splitter.split_text(text)
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-V2")
embeddings = model.encode(chunks)
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(np.array(embeddings))

def search_faiss(query):
    q_embed = model.encode([query])
    _, idx = index.search(np.array(q_embed), k=1)
    return chunks[idx[0][0]]

prompt = PromptTemplate(
    input_variables=["ipc_section", "query"],
    template="""You are an expert in Indian law. A user asked: "{query}"
Based on the Indian Penal Code (IPC), the relevant section is:
{ipc_section}
Please provide:
- A simple explanation
- The key legal points
- Possible punishments
- A real-world example"""
)

def ipc_bot(query):
    section = search_faiss(query)
    return LLMChain(llm=llm, prompt=prompt).run(ipc_section=section, query=query)

print(ipc_bot("What is the punishment for theft under IPC?"))